In [8]:
print("Ok")

Ok


In [9]:
%pwd

'c:\\Users\\tiyas\\Desktop\\My learning (Data Science)\\Projects\\Medibot\\research'

In [10]:
import os
#os.chdir(r"My learning (Data Science)\Projects\Medibot")
os.chdir("../")

In [11]:
%pwd

'c:\\Users\\tiyas\\Desktop\\My learning (Data Science)\\Projects\\Medibot'

In [12]:
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [13]:
# Extract data from the pdf file
def load_pdf_file(data):
    loader = DirectoryLoader(data,
                             glob = "*.pdf",
                             loader_cls= PyPDFLoader)
    
    documents = loader.load()

    return documents

In [14]:
extracted_data = load_pdf_file(data = 'Data/')

In [15]:
# extracted_data

In [16]:
# Split data into text chunks
def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
    text_chunks = text_splitter.split_documents(extracted_data)
    return text_chunks

In [17]:
text_chunks = text_split(extracted_data)
print("Length of chunks:", len(text_chunks))

Length of chunks: 686


In [18]:
#text_chunks

In [19]:
from langchain.embeddings import HuggingFaceEmbeddings
from sentence_transformers import SentenceTransformer


c:\Users\tiyas\anaconda3\envs\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [20]:
# Download embeddings from huggingface
def download_hugging_face_embeddings():
    embeddings = HuggingFaceEmbeddings(model_name = 'sentence-transformers/all-MiniLM-L6-v2')
    return embeddings

In [21]:
embeddings = download_hugging_face_embeddings()

C:\Users\tiyas\AppData\Local\Temp\ipykernel_33740\1220083666.py:3: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name = 'sentence-transformers/all-MiniLM-L6-v2')


In [22]:
query_result = embeddings.embed_query("Hello World")
print("Length", len(query_result))

Length 384


In [35]:
from dotenv import load_dotenv
load_dotenv()

True

In [36]:
PINECONE_API_KEY = os.environ.get('PINECONE_API_KEY')
OPENAI_API_KEY = os.environ.get('OPENAI_API_KEY')

In [28]:
# Create Pinecone Index
import os
from pinecone import Pinecone, ServerlessSpec

# Initialize Pinecone client
pc = Pinecone(api_key=PINECONE_API_KEY)

# Set a valid index name
index_name = "medibot"  

# Create the index only if it doesn't exist
if index_name not in pc.list_indexes().names():
    pc.create_index(
        name=index_name,
        dimension=384,  
        metric="cosine",
        spec=ServerlessSpec(
            cloud="aws",
            region="us-east-1"
        )
    )
    print(f"Index '{index_name}' created successfully.")
else:
    print(f"Index '{index_name}' already exists.")


Index 'medibot' created successfully.


In [37]:
import os
os.environ["PINECONE_API_KEY"] = PINECONE_API_KEY
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

In [30]:
# Embed each chunk and insert the embeddings into Pinecone index
from langchain_pinecone import PineconeVectorStore

docsearch = PineconeVectorStore.from_documents(
    documents=text_chunks,
    index_name=index_name,
    embedding=embeddings
)

In [31]:
# Load existing index

from langchain_pinecone import PineconeVectorStore

docsearch = PineconeVectorStore.from_existing_index(
    index_name=index_name,
    embedding=embeddings
)

In [32]:
docsearch

In [33]:
retriever = docsearch.as_retriever(search_type='similarity', search_kwargs={'k':3})

In [34]:
retrieved_docs = retriever.invoke("What is cancer?")
retrieved_docs

[Document(id='4061f250-6f35-4904-b775-ca99d1d40683', metadata={'creationdate': '2020-05-28T11:24:01+05:30', 'creator': 'QuarkXPressª 4.1: LaserWriter 8 8.6', 'keywords': '"brochures, publications, breast cancer"', 'moddate': '2020-11-19T16:58:11-08:00', 'page': 27.0, 'page_label': '28', 'pdfversion': '1.4', 'producer': '3-Heights(TM) PDF Security Shell 4.8.25.2 (http://www.pdf-tools.com)', 'source': 'Data\\breast_cancer.pdf', 'title': "A Woman's Guide to Breast Cancer Diagnosis & Treatment", 'total_pages': 32.0}, page_content='Malignant: cancer.\nMammogram: an x ray of the breast. \nMastectomy: removing the breast by surgery.\nMetastasis: spread of cancer from onepart of the body to another.'),
 Document(id='8cdd18f2-d900-4b4a-8e6f-1c69ade678bf', metadata={'creationdate': '2020-05-28T11:24:01+05:30', 'creator': 'QuarkXPressª 4.1: LaserWriter 8 8.6', 'keywords': '"brochures, publications, breast cancer"', 'moddate': '2020-11-19T16:58:11-08:00', 'page': 23.0, 'page_label': '24', 'pdfvers

In [38]:
from langchain import OpenAI
llm = OpenAI(temperature = 0.4, max_tokens = 500)

C:\Users\tiyas\AppData\Local\Temp\ipykernel_33740\2685721183.py:2: LangChainDeprecationWarning: The class `OpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAI``.
  llm = OpenAI(temperature = 0.4, max_tokens = 500)


In [39]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, then say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}")
    ]
)

In [40]:
question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

In [41]:
response = rag_chain.invoke({"input" : "What is cancer?"} )
print(response["answer"])



Cancer is a disease caused by the uncontrolled growth and spread of abnormal cells in the body. It can be classified as either benign or malignant, with malignant cancer cells being capable of spreading to other parts of the body. Cancer can be detected through medical imaging techniques such as mammograms and can be treated through various methods, including surgery.
